# Primary analyses (manuscript)

> **Note**
>
> In this notebook, we record the analyses that are reported in the main body of the manuscript.
>
> The section [Save results for manuscript](primary_analyses.qmd#sec-saveresults) lists all results objects and tables that are exported to be used in the main manuscript file and can be used to retrace all reported numbers.

## Load packages

In [ ]:
library(foreign, quietly = TRUE, warn.conflicts = FALSE)
library(tidyverse, quietly = TRUE, warn.conflicts = FALSE)


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.
 
###############################################################################
This is semTools 0.5-7
All users of R (or SEM) are invited to submit functions or ideas for functions.
###############################################################################

In [ ]:
# record R packages
papaja::r_refs(file = "../r-references.bib")


## Load datasets

In [ ]:
dat_poly <- read.spss("../data/Haupterhebung Studie III (Polytom 6PS).sav", 
  to.data.frame = TRUE, use.value.labels = FALSE, na.omit = FALSE)
dat_dich <- read.spss("../data/Haupterhebung Studie III (Dichotom 2PS).sav",
  to.data.frame = TRUE, use.value.labels = FALSE, na.omit = FALSE)


## Descriptive statistics

In [ ]:
dat_desc <- bind_rows(
  dat_poly |> select(Alter_realW, Geschlecht, Bildung, Muttersprache) |> mutate(categories = "6"),
  dat_dich |> select(Alter_realW, Geschlecht,Bildung, Muttersprache) |> mutate(categories = "2")
)

table1 <- dat_desc |>
  rename(Gender = Geschlecht, Education = Bildung, `Mother tongue` = Muttersprache, Age = Alter_realW) |>
  mutate(categories = factor(categories, levels = c(2, 6), labels = c("Two categories", "Six categories")),
    Gender = factor(Gender, levels = 1:2, labels = c("Women", "Men")), 
    Education = factor(Education, levels = 1:6, labels = c("No school leaving certificate", "Secondary school leaving certificate/elementary school or equivalent", "Secondary school or equivalent", "Vocational baccalaureate or high school diploma", "College degree or university degree", "Doctorate or habilitation")), 
    `Mother tongue` = factor(`Mother tongue`, levels = 1:2, labels = c("German", "Other"))) |>
  flextable::summarizor(by = "categories") |>
  flextable::as_flextable() |>
  flextable::theme_apa() |>
  ftExtra::colformat_md()


Registered S3 method overwritten by 'ftExtra':
  method                  from     
  as_flextable.data.frame flextable

Two categories (N=921) Six categories (N=933) Age Mean (SD) 32.8 (15.9) 32.6 (16.1) Median (IQR) 25.0 (23.0) 25.0 (21.0) Range 18.0 - 89.0 18.0 - 97.0 Gender Women 512 (55.6%) 524 (56.2%) Men 409 (44.4%) 409 (43.8%) Education No school leaving certificate 1 (0.1%) 2 (0.2%) Secondary school leaving certificate/elementary school or equivalent 35 (3.8%) 43 (4.6%) Secondary school or equivalent 129 (14.0%) 119 (12.8%) Vocational baccalaureate or high school diploma 446 (48.4%) 437 (46.8%) College degree or university degree 280 (30.4%) 311 (33.3%) Doctorate or habilitation 30 (3.3%) 21 (2.3%) Mother tongue German 837 (90.9%) 860 (92.2%) Other 84 (9.1%) 73 (7.8%)

## Fit and analyze CFA models

### Tau-congeneric model for each trait

In [ ]:
### tau-congeneric model without modification 
modelAP <- "
  f1 =~ A_1 +  A_2 + A_3 + A_4 + A_5 + A_6 + A_7 + A_8 + A_9 +A_10 + A_11 + A_12 + Alter_realW"
modelGP <- "
  f1 =~ KGr_13 + KGr_14 + KGr_15 + KGr_16 + KGr_17 + KGr_18 + KGr_19 + KGr_20 + KGr_21 + KGr_22 + KGr_23 + KGr_24 + Koerpergroesse_realW"
modelGeP <- "
  f1 =~ KGe_25 + KGe_26 + KGe_27 + KGe_28 + KGe_29 + KGe_30 + KGe_31 + KGe_32 + KGe_33 + KGe_34 + KGe_35 + KGe_36 + Koerpergewicht_realW"


fit1A6P <- cfa(model = modelAP, data = dat_poly, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12"), std.lv=TRUE, meanstructure=TRUE)
fit1A2P <- cfa(model = modelAP, data = dat_dich, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12"), std.lv=TRUE, meanstructure=TRUE)
fit1G6P <- cfa(model = modelGP, data = dat_poly, ordered = c("KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24"), std.lv=TRUE, meanstructure=TRUE)
fit1G2P <- cfa(model = modelGP, data = dat_dich, ordered = c("KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24"), std.lv=TRUE, meanstructure=TRUE)
fit1Ge6P <- cfa(model = modelGeP, data = dat_poly, ordered = c("KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)
fit1Ge2P <- cfa(model = modelGeP, data = dat_dich, ordered = c("KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)

summary(fit1A6P, fit.measures=TRUE, standardized = TRUE, rsquare = TRUE, modindices = TRUE)


lavaan 0.6-19 ended normally after 24 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        75

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                              1208.558    1647.923
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.743
  Shift parameter                                           20.880
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             20936.426    9407.573
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 21 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        27

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               496.587     690.371
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.734
  Shift parameter                                           14.198
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                              9520.005    5306.474
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 32 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        75

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               605.167    1113.389
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.555
  Shift parameter                                           22.668
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             52160.203   16417.273
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 25 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        27

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               318.218     423.713
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.787
  Shift parameter                                           19.480
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             25383.952   12259.663
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 32 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        75

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               556.239    1003.750
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.562
  Shift parameter                                           14.732
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             45552.959   18030.793
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 24 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        27

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               280.194     394.701
  Degrees of freedom                                65          65
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.734
  Shift parameter                                           12.769
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             11668.046    6695.698
  Degrees of freedom                                78          78
  P-value                                        0.000       0.000
  Scal

####################### Model Fit Indices ###########################
         chisq.scaled df.scaled pvalue.scaled rmsea.scaled cfi.scaled
fit1A6P     1647.923         65          .000        .162       .830 
fit1G6P     1113.389         65          .000        .132       .936 
fit1Ge6P    1003.750†        65          .000        .124†      .948†
         tli.scaled  srmr
fit1A6P       .796  .105 
fit1G6P       .923  .069 
fit1Ge6P      .937† .069†

####################### Model Fit Indices ###########################
         chisq.scaled df.scaled pvalue.scaled rmsea.scaled cfi.scaled
fit1A2P      690.371         65          .000        .102       .880 
fit1G2P      423.713         65          .000        .077       .971†
fit1Ge2P     394.701†        65          .000        .074†      .950 
         tli.scaled  srmr
fit1A2P       .856  .104 
fit1G2P       .965† .091 
fit1Ge2P      .940  .086†

In [ ]:
load_df_nomod <- 
  bind_rows(
    standardizedSolution(fit1A6P, type = "std.all") |> mutate(measure = "age", categories = "6"),
    standardizedSolution(fit1A2P, type = "std.all") |> mutate(measure = "age", categories = "2"),
    standardizedSolution(fit1G6P, type = "std.all") |> mutate(measure = "height", categories = "6"),
    standardizedSolution(fit1G2P, type = "std.all") |> mutate(measure = "height", categories = "2"),
    standardizedSolution(fit1Ge6P, type = "std.all") |> mutate(measure = "weight", categories = "6"),
    standardizedSolution(fit1Ge2P, type = "std.all") |> mutate(measure = "weight", categories = "2")) |>
  filter(lhs == "f1" & op == "=~") |> 
  mutate(item_name = rhs)

load_df_nomod_proc <- load_df_nomod |>
  # Create new columns: prefix (everything before underscore) and suffix (the numeric part)
  mutate(prefix = sub("_.*", "", item_name),
    suffix = as.numeric(sub(".*_", "", item_name))) |>
  group_by(prefix) |>
  mutate(item = match(suffix, sort(unique(suffix)))) |>
  mutate(item = replace_na(as.character(item), "Physical item")) |>
  ungroup() |>
  mutate(est_print = papaja::printnum(est.std, gt1 = FALSE, digits = 3)) |>
  select(item, measure, categories, est_print) |>
  pivot_wider(names_from = c(measure, categories), values_from = est_print)


ℹ In argument: `suffix = as.numeric(sub(".*_", "", item_name))`.
Caused by warning:
! NAs introduced by coercion

ℹ In argument: `est_print = papaja::printnum(est, gt1 = FALSE, digits = 3)`.
Caused by warning in `apa_num.numeric()`:
! You specified gt1 = FALSE, but passed absolute value(s) that exceed 1.

The first warning was:
ℹ In argument: `est_print = if_else(...)`.
Caused by warning in `validate()`:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 1 remaining warning.

Measure Height Weight Age Categories Two Six Two Six Two Six Items 1 .780 .611 .564 .564 .430 .493 2 .816 .829 .456 .503 .668 .756 3 .786 .740 .853 .822 .671 .535 4 .883 .851 .931 .941 .764 .738 5 .778 .742 .949 .902 .786 .774 6 .877 .817 .886 .897 .129 .230 7 .843 .808 -.778 -.749 .921 .837 8 .623 .721 -.649 -.619 .586 .571 9 -.887 -.835 -.420 -.421 .795 .700 10 -.843 -.719 .613 .683 -.810 -.681 11 -.877 -.818 .245 .261 .453 .489 12 .876 .815 .696 .751 .698 .739 Physical item .873 .863 .698 .690 .759 .719 Model fit χ 2 423.713 1,113.389 394.701 1,003.750 69.371 1,647.923 df 65 65 65 65 65 65 p < .001 < .001 < .001 < .001 < .001 < .001 CFI .971 .936 .950 .948 .880 .830 RMSEA .077 .132 .074 .124 .102 .162 SRMR .091 .069 .086 .069 .104 .105

### Bifactor models

In [ ]:
### Residual Psychologie
modelA_p <- "
  age =~ A_1 + A_2 + A_3 + A_4 + A_5 + A_6 + A_7 + A_8 + A_9 + A_10 + A_11 + A_12
  A_1 ~ Alter_realW + Geschlecht 
  A_2 ~ Alter_realW + Geschlecht
  A_3 ~ Alter_realW + Geschlecht
  A_4 ~ Alter_realW + Geschlecht
  A_5 ~ Alter_realW + Geschlecht
  A_6 ~ Alter_realW + Geschlecht
  A_7 ~ Alter_realW + Geschlecht
  A_8 ~ Alter_realW + Geschlecht
  A_9 ~ Alter_realW + Geschlecht
  A_10 ~ Alter_realW + Geschlecht
  A_11 ~ Alter_realW + Geschlecht
  A_12 ~ Alter_realW + Geschlecht
"

modelG_p <- "
height =~ KGr_13 + KGr_14 + KGr_15 + KGr_16 + KGr_17 + KGr_18 + KGr_19 + KGr_20 + KGr_21 + KGr_22 + KGr_23 + KGr_24
  KGr_13 ~ Koerpergroesse_realW + Geschlecht
  KGr_14 ~ Koerpergroesse_realW + Geschlecht
  KGr_15 ~ Koerpergroesse_realW + Geschlecht
  KGr_16 ~ Koerpergroesse_realW + Geschlecht
  KGr_17 ~ Koerpergroesse_realW + Geschlecht
  KGr_18 ~ Koerpergroesse_realW + Geschlecht
  KGr_19 ~ Koerpergroesse_realW + Geschlecht
  KGr_20 ~ Koerpergroesse_realW + Geschlecht
  KGr_21 ~ Koerpergroesse_realW + Geschlecht
  KGr_22 ~ Koerpergroesse_realW + Geschlecht
  KGr_23 ~ Koerpergroesse_realW + Geschlecht
  KGr_24 ~ Koerpergroesse_realW + Geschlecht
"
modelGe_p <- "
  weight =~ KGe_25 + KGe_26 + KGe_27 + KGe_28 + KGe_29 + KGe_30 + KGe_31 + KGe_32 + KGe_33 + KGe_34 + KGe_35 + KGe_36
  KGe_25 ~ Koerpergewicht_realW + Geschlecht
  KGe_26 ~ Koerpergewicht_realW + Geschlecht
  KGe_27 ~ Koerpergewicht_realW + Geschlecht
  KGe_28 ~ Koerpergewicht_realW + Geschlecht
  KGe_29 ~ Koerpergewicht_realW + Geschlecht
  KGe_30 ~ Koerpergewicht_realW + Geschlecht
  KGe_31 ~ Koerpergewicht_realW + Geschlecht
  KGe_32 ~ Koerpergewicht_realW + Geschlecht
  KGe_33 ~ Koerpergewicht_realW + Geschlecht
  KGe_34 ~ Koerpergewicht_realW + Geschlecht
  KGe_35 ~ Koerpergewicht_realW + Geschlecht
  KGe_36 ~ Koerpergewicht_realW + Geschlecht
"

fit1A6_p <- cfa(model = modelA_p, data = dat_poly, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12"), std.lv=TRUE, meanstructure=TRUE)
fit1A2_p <- cfa(model = modelA_p, data = dat_dich, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12"), std.lv=TRUE, meanstructure=TRUE)
fit1G6_p <- cfa(model = modelG_p, data = dat_poly, ordered = c("KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24"), std.lv=TRUE, meanstructure=TRUE)
fit1G2_p <- cfa(model = modelG_p, data = dat_dich, ordered = c("KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24"), std.lv=TRUE, meanstructure=TRUE)
fit1Ge6_p <- cfa(model = modelGe_p, data = dat_poly, ordered = c("KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)
fit1Ge2_p <- cfa(model = modelGe_p, data = dat_dich, ordered = c("KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)

summary(fit1A6_p, fit.measures=TRUE, standardized = TRUE, rsquare = TRUE, modindices = TRUE)


lavaan 0.6-19 ended normally after 84 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        96

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               348.616     465.948
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.761
  Shift parameter                                            7.879
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                              7464.840    4843.443
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 85 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        48

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               171.791     198.298
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.898
  Shift parameter                                            7.013
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                              1716.483    1368.521
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 67 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        96

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               982.976    1063.184
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.936
  Shift parameter                                           12.688
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                              4039.107    2693.986
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 61 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        48

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               381.228     336.922
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.182
  Shift parameter                                           14.481
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                               962.199     726.289
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 93 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        96

  Number of observations                           933

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               425.582     571.129
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.756
  Shift parameter                                            8.342
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             10231.376    6364.399
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

lavaan 0.6-19 ended normally after 87 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        48

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                               231.873     267.896
  Degrees of freedom                                54          54
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  0.894
  Shift parameter                                            8.592
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                              2118.911    1537.384
  Degrees of freedom                                66          66
  P-value                                        0.000       0.000
  Scal

In [ ]:
load_df_bifac <- 
  bind_rows(
    standardizedSolution(fit1A6_p, type = "std.all") |> mutate(measure = "age", categories = "6"),
    standardizedSolution(fit1A2_p, type = "std.all") |> mutate(measure = "age", categories = "2"),
    standardizedSolution(fit1G6_p, type = "std.all") |> mutate(measure = "height", categories = "6"),
    standardizedSolution(fit1G2_p, type = "std.all") |> mutate(measure = "height", categories = "2"),
    standardizedSolution(fit1Ge6_p, type = "std.all") |> mutate(measure = "weight", categories = "6"),
    standardizedSolution(fit1Ge2_p, type = "std.all") |> mutate(measure = "weight", categories = "2")) |>
  filter(
    (lhs %in% c("age", "height", "weight") & op == "=~") |
    (rhs %in% c("Alter_realW", "Koerpergroesse_realW", "Koerpergewicht_realW", "Geschlecht") & op == "~"))
  
load_df_bifac_proc <- load_df_bifac |>
  mutate(
    cause = case_when(
      lhs %in% c("age", "height", "weight") & op == "=~" ~ lhs,
      rhs %in% c("Alter_realW", "Koerpergroesse_realW", "Koerpergewicht_realW", "Geschlecht") & op == "~" ~ rhs),
    item_name = case_when(
      lhs %in% c("age", "height", "weight") & op == "=~" ~ rhs,
      rhs %in% c("Alter_realW", "Koerpergroesse_realW", "Koerpergewicht_realW", "Geschlecht") & op == "~" ~ lhs
    )) |> 
  # Create new columns: prefix (everything before underscore) and suffix (the numeric part)
  mutate(prefix = sub("_.*", "", item_name),
    suffix = as.numeric(sub(".*_", "", item_name))) |>
  # For each prefix group, assign new_num as the rank of suffix in sorted order 
  group_by(prefix) |>
  mutate(item = match(suffix, sort(unique(suffix)))) |>
  ungroup() |>
  mutate(est_print = papaja::printnum(est.std, gt1 = TRUE, digits = 3))


In [ ]:
table3 <- load_df_bifac_proc |> 
  select(item, cause, measure, categories, est_print) |> 
  pivot_wider(names_from = c(measure, cause), values_from = est_print) |>
  select(item, categories, height_height, height_Koerpergroesse_realW, height_Geschlecht, weight_weight, weight_Koerpergewicht_realW, weight_Geschlecht, age_age, age_Alter_realW, age_Geschlecht) |>
  mutate(categories = factor(categories, levels = c(2, 6), labels = c("Two categories", "Six categories"))) |>
  arrange(categories, item) |>
  flextable::as_grouped_data(groups = "categories") |>
  flextable::as_flextable(hide_grouplabel = TRUE) |>
  flextable::add_header_row(values = c("", "Height", "Weight", "Age"),
    colwidths = c(1, 3, 3, 3)) |>
  flextable::set_header_labels(
    values = list(
      item = "Item",
      age_age = "Latent", height_height = "Latent", weight_weight = "Latent",
      age_Alter_realW = "Physical",
      height_Koerpergroesse_realW = "Physical",
      weight_Koerpergewicht_realW = "Physical",
      age_Geschlecht = "Gender", height_Geschlecht = "Gender", weight_Geschlecht = "Gender")) |>
  flextable::align(i = 1, align = "center", part = "header") |>
  flextable::border(i = 2, j = 1, border.top = officer::fp_border(width = 0), part = "header") |>
  flextable::border(i = 1, j = 1, border.bottom = officer::fp_border(width = 0), part = "header") |>
  
  flextable::bold(i = ~ abs(parse_number(age_age)) > 0.6, j = "age_age") |>
  flextable::bold(i = ~ abs(parse_number(age_Alter_realW)) > 0.6, j = "age_Alter_realW") |>
  flextable::bold(i = ~ abs(parse_number(age_Geschlecht)) > 0.6, j = "age_Geschlecht") |>
  flextable::bold(i = ~ abs(parse_number(height_height)) > 0.6, j = "height_height") |>
  flextable::bold(i = ~ abs(parse_number(height_Koerpergroesse_realW)) > 0.6, j = "height_Koerpergroesse_realW") |>
  flextable::bold(i = ~ abs(parse_number(height_Geschlecht)) > 0.6, j = "height_Geschlecht") |>
  flextable::bold(i = ~ abs(parse_number(weight_weight)) > 0.6, j = "weight_weight") |>
  flextable::bold(i = ~ abs(parse_number(weight_Koerpergewicht_realW)) > 0.6, j = "weight_Koerpergewicht_realW") |>
  flextable::bold(i = ~ abs(parse_number(weight_Geschlecht)) > 0.6, j = "weight_Geschlecht") |>
  
  flextable::theme_apa() |>
  ftExtra::colformat_md()
table3


Height Weight Age Item Latent Physical Gender Latent Physical Gender Latent Physical Gender Two categories 1 0.223 0.804 -0.138 0.241 0.452 -0.043 0.336 0.287 -0.058 2 0.368 0.882 -0.274 0.441 0.313 -0.185 0.795 0.278 -0.071 3 0.405 0.922 -0.475 0.478 0.764 -0.357 0.015 0.655 0.077 4 0.292 0.895 -0.114 0.682 0.740 -0.278 0.637 0.481 -0.024 5 0.346 0.742 -0.067 0.574 0.877 -0.346 0.754 0.454 -0.056 6 0.312 0.955 -0.279 0.652 0.796 -0.417 0.271 -0.005 -0.004 7 0.299 0.922 -0.231 -0.385 -0.767 0.265 0.270 0.797 -0.043 8 0.207 0.972 -0.778 -0.246 -0.627 0.155 0.377 0.405 -0.246 9 -0.319 -0.883 0.088 -0.020 -0.505 0.095 0.024 0.777 0.094 10 -0.361 -0.917 0.270 0.263 0.468 0.070 -0.265 -0.687 0.017 11 -0.303 -1.061 0.399 0.174 0.143 0.086 0.292 0.333 -0.016 12 0.318 1.011 -0.373 0.634 0.561 -0.400 0.577 0.448 0.036 Six categories 1 0.223 0.624 -0.116 0.362 0.432 -0.122 0.458 0.258 0.053 2 0.365 0.869 -0.197 0.413 0.387 -0.259 0.755 0.327 0.041 3 0.408 0.701 -0.147 0.475 0.759 -0.315 0.029 0.583 0.059 4 0.391 0.863 -0.169 0.643 0.788 -0.445 0.700 0.367 0.051 5 0.306 0.695 -0.016 0.536 0.846 -0.403 0.698 0.409 0.016 6 0.363 0.834 -0.174 0.632 0.756 -0.477 0.273 0.087 0.014 7 0.369 0.830 -0.184 -0.396 -0.755 0.374 0.302 0.762 0.031 8 0.355 0.899 -0.484 -0.277 -0.627 0.194 0.426 0.395 -0.121 9 -0.382 -0.801 0.099 -0.114 -0.464 0.043 0.036 0.754 0.053 10 -0.291 -0.790 0.248 0.331 0.625 -0.045 -0.201 -0.664 -0.048 11 -0.323 -0.949 0.340 0.187 0.210 -0.065 0.443 0.270 -0.059 12 0.321 0.909 -0.273 0.577 0.591 -0.404 0.613 0.447 0.112

### Full model

In [ ]:
modelges <- "
  age =~ A_1 +  A_2 + A_3 + A_4 + A_5 + A_6 + A_7 + A_8 + A_9 + A_10 + A_11 + A_12 
  height =~ KGr_13 + KGr_14 + KGr_15 + KGr_16 + KGr_17 + KGr_18 + KGr_19 + KGr_20 + KGr_21 + KGr_22 + KGr_23 + KGr_24 
  weight =~ KGe_25 + KGe_26 + KGe_27 + KGe_28 + KGe_29 + KGe_30 + KGe_31 + KGe_32 + KGe_33 + KGe_34 + KGe_35 + KGe_36
  Alter_realW ~~ Koerpergroesse_realW
  Koerpergewicht_realW ~~ Alter_realW
  Koerpergroesse_realW ~~ Koerpergewicht_realW

  Alter_realW ~~ age
  Koerpergewicht_realW ~~ weight
  Koerpergroesse_realW ~~ height
" 

fit1ges6 <- cfa(model = modelges, data = dat_poly, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12", "KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24", "KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)
fit1ges2 <- cfa(model = modelges, data = dat_dich, ordered = c("A_1","A_2", "A_3", "A_4", "A_5", "A_6", "A_7","A_8", "A_9", "A_10", "A_11","A_12", "KGr_13","KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19","KGr_20", "KGr_21", "KGr_22", "KGr_23","KGr_24", "KGe_25","KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31","KGe_32", "KGe_33", "KGe_34", "KGe_35","KGe_36"), std.lv=TRUE, meanstructure=TRUE)

summary(fit1ges6, fit.measures=TRUE, standardized = TRUE, rsquare = TRUE, modindices = TRUE)


lavaan 0.6-19 ended normally after 119 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                       231

  Number of observations                           933

Model Test User Model:
                                               Standard      Scaled
  Test Statistic                              10068.719    6203.190
  Degrees of freedom                                696         696
  P-value (Chi-square)                            0.000       0.000
  Scaling correction factor                                   1.739
  Shift parameter                                           414.041
    simple second-order correction                                 

Model Test Baseline Model:

  Test statistic                            131319.540   37797.377
  Degrees of freedom                               741         741
  P-value                                        0.000       0.00

lavaan 0.6-19 ended normally after 104 iterations

  Estimator                                       DWLS
  Optimization method                           NLMINB
  Number of model parameters                        87

  Number of observations                           921

Model Test User Model:
                                              Standard      Scaled
  Test Statistic                              5210.929    3440.015
  Degrees of freedom                               696         696
  P-value (Chi-square)                           0.000       0.000
  Scaling correction factor                                  1.698
  Shift parameter                                          370.538
    simple second-order correction                                

Model Test Baseline Model:

  Test statistic                             52572.698   20040.141
  Degrees of freedom                               741         741
  P-value                                        0.000       0.000
  Sca

## Correlations between latent variables

In [ ]:
(r_latent_poly <- standardizedSolution(fit1ges6, type = "std.all") |>
  filter(op == "~~" & 
      lhs %in% c("age", "height", "weight") &
      rhs %in% c("age", "height", "weight") &
      lhs != rhs) |>
  mutate(est_print = papaja::printnum(est.std, gt1 = FALSE, digits = 3)) |>
  mutate(est_print_star = case_when(
    pvalue < 0.05 & pvalue > 0.01 ~ paste0(est_print, "*"),
    pvalue < 0.01 & pvalue > 0.001 ~ paste0(est_print, "**"),
    pvalue < 0.001 ~ paste0(est_print, "***"),
    .default = est_print 
  )))


     lhs op    rhs est.std    se      z pvalue ci.lower ci.upper est_print
1    age ~~ height   0.003 0.032  0.092  0.927   -0.060    0.066      .003
2    age ~~ weight   0.371 0.029 12.693  0.000    0.314    0.428      .371
3 height ~~ weight   0.182 0.031  5.771  0.000    0.120    0.243      .182
  est_print_star
1           .003
2        .371***
3        .182***

     lhs op    rhs est.std    se      z pvalue ci.lower ci.upper est_print
1    age ~~ height  -0.053 0.038 -1.383  0.167   -0.128    0.022     -.053
2    age ~~ weight   0.297 0.041  7.254  0.000    0.217    0.378      .297
3 height ~~ weight   0.252 0.039  6.420  0.000    0.175    0.329      .252
  est_print_star
1          -.053
2        .297***
3        .252***

## Correlations between physical variables

In [ ]:
(r_phys_poly <- standardizedSolution(fit1ges6, type = "std.all") |>
  filter(op == "~~" & 
      lhs %in% c("Alter_realW", "Koerpergroesse_realW", "Koerpergewicht_realW") &
      rhs %in% c("Alter_realW", "Koerpergroesse_realW", "Koerpergewicht_realW") &
      lhs != rhs) |>
  mutate(est_print = papaja::printnum(est.std, gt1 = FALSE, digits = 3)) |>
  mutate(est_print_star = case_when(
    pvalue < 0.05 & pvalue > 0.01 ~ paste0(est_print, "*"),
    pvalue < 0.01 & pvalue > 0.001 ~ paste0(est_print, "**"),
    pvalue < 0.001 ~ paste0(est_print, "***"),
    .default = est_print 
  )))


                   lhs op                  rhs est.std    se      z pvalue
1          Alter_realW ~~ Koerpergroesse_realW  -0.116 0.033 -3.498      0
2          Alter_realW ~~ Koerpergewicht_realW   0.204 0.031  6.587      0
3 Koerpergewicht_realW ~~ Koerpergroesse_realW   0.589 0.011 54.213      0
  ci.lower ci.upper est_print est_print_star
1   -0.181   -0.051     -.116       -.116***
2    0.143    0.265      .204        .204***
3    0.568    0.611      .589        .589***

                   lhs op                  rhs est.std    se      z pvalue
1          Alter_realW ~~ Koerpergroesse_realW  -0.089 0.033 -2.665  0.008
2          Alter_realW ~~ Koerpergewicht_realW   0.185 0.032  5.793  0.000
3 Koerpergewicht_realW ~~ Koerpergroesse_realW   0.658 0.014 47.086  0.000
  ci.lower ci.upper est_print est_print_star
1   -0.154   -0.023     -.089        -.089**
2    0.123    0.248      .185        .185***
3    0.630    0.685      .658        .658***

## Reliability estimates

In [ ]:
## ------------------------------------------------------------
## 1. recode inverse items on scales from 1-6 and 0-1
## ------------------------------------------------------------

# recode 6-point scale
recode_inverse_6 <- c('1' = 6L, '2' = 5L, '3' = 4L, '4' = 3L, '5' = 2L, '6' = 1L)

dat_poly_r <- dat_poly |>
  mutate(across(all_of(c("A_10", "KGr_21", "KGr_22", "KGr_23", "KGe_31", "KGe_32", "KGe_33")),
                ~ recode(.x, !!!recode_inverse_6)))

# recode 2-point scale
recode_inverse_2 <- c('1' = 0L, '0' = 1L)

dat_dich_r <- dat_dich |>
  mutate(across(all_of(c("A_10", "KGr_21", "KGr_22", "KGr_23", "KGe_31", "KGe_32", "KGe_33")),
                ~ recode(.x, !!!recode_inverse_2)))

## ------------------------------------------------------------
## 2. define item sets
## ------------------------------------------------------------

items_age <- c("A_1", "A_2", "A_3", "A_4", "A_5", "A_6", "A_7", "A_8", "A_9", "A_10", "A_11", "A_12")
items_height <- c("KGr_13", "KGr_14", "KGr_15", "KGr_16", "KGr_17", "KGr_18", "KGr_19", "KGr_20", "KGr_21", "KGr_22", "KGr_23", "KGr_24")
items_weight <- c("KGe_25", "KGe_26", "KGe_27", "KGe_28", "KGe_29", "KGe_30", "KGe_31", "KGe_32", "KGe_33", "KGe_34", "KGe_35", "KGe_36")

## ------------------------------------------------------------
## 3. Reliabilitätsanalyse (McDonald's Omega) mit Konfidenzintervallen
## ------------------------------------------------------------

# Für dat_poly (6-stufige Items)
(rel_A6 <- ci.reliability(data = dat_poly_r[, items_age], type = "omega", interval.type = "ml"))


$est
[1] 0.8547127

$se
[1] 0.00706836

$ci.lower
[1] 0.840859

$ci.upper
[1] 0.8685664

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

$est
[1] 0.9199587

$se
[1] 0.003874887

$ci.lower
[1] 0.9123641

$ci.upper
[1] 0.9275534

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

$est
[1] 0.8857536

$se
[1] 0.005546829

$ci.lower
[1] 0.874882

$ci.upper
[1] 0.8966252

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

$est
[1] 0.793557

$se
[1] 0.01003342

$ci.lower
[1] 0.7738918

$ci.upper
[1] 0.8132221

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

$est
[1] 0.8778504

$se
[1] 0.005974573

$ci.lower
[1] 0.8661404

$ci.upper
[1] 0.8895603

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

$est
[1] 0.7854144

$se
[1] 0.01050752

$ci.lower
[1] 0.7648201

$ci.upper
[1] 0.8060088

$conf.level
[1] 0.95

$type
[1] "omega"

$interval.type
[1] "maximum likelihood (wald ci)"

In [ ]:
cor_table <- tribble(
  ~Measure, ~Height_2  , ~Height_6  , ~Weight_2    , ~Weight_6    , ~Age_2       , ~Age_6       ,
  "Height", rel_G2     , rel_G6     , r_latent_GGe2, r_latent_GGe6, r_latent_AG2 , r_latent_AG6 ,
  "Weight", r_phys_GGe2, r_phys_GGe6, rel_Ge2      , rel_Ge6      , r_latent_AGe2, r_latent_AGe6,
  "Age"   , r_phys_AG2 , r_phys_AG6 , r_phys_AGe2  , r_phys_AGe6  , rel_A2       , rel_A6      
)


In [ ]:
table4 <- flextable::flextable(cor_table) |>
  flextable::set_header_labels(
    Measure = "Categories",
    Age_6 = "Six", Age_2 = "Two",
    Height_6 = "Six", Height_2 = "Two",
    Weight_6 = "Six", Weight_2 = "Two") |>
  flextable::add_header_row(values = c("Measure", "Height", "Weight", "Age"),
    colwidths = c(1, 2, 2, 2)) |>
  flextable::align(i = 1, align = "center", part = "header") |>
  flextable::border(i = 2, j = 1, border.top = officer::fp_border(width = 0), part = "header") |>
  flextable::border(i = 1, j = 1, border.bottom = officer::fp_border(width = 0), part = "header") |>
  
  flextable::bold(i  = 1, j = 4:7) |>
  flextable::bold(i  = 2, j = 6:7) |>

  flextable::bg(i  = 1, j = c(2, 3), bg = "grey90") |>
  flextable::bg(i  = 2, j = c(4, 5), bg = "grey90") |>
  flextable::bg(i  = 3, j = c(6, 7), bg = "grey90") |>
  
  flextable::theme_apa() |>
  ftExtra::colformat_md()
table4


Measure Height Weight Age Categories Two Six Two Six Two Six Height .878 .920 .252*** .182*** -.053 .003 Weight .658*** .589*** .785 .886 .297*** .371*** Age -.089** -.116*** .185*** .204*** .794 .855

## Save results for manuscript

In [ ]:
results <- saveRDS(
  object = list(
    table1 = table1, table2 = table2, table3 = table3, table4 = table4,
    load_df_nomod = load_df_nomod,
    load_df_bifac = load_df_bifac,
    fullmodelfit6 = fitMeasures(fit1ges6),
    fullmodelfit2 = fitMeasures(fit1ges2)
    ), 
  file = "../results/results.rds"
)
